# Imports, load data , training functions and database creation

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, errno
import networkx as nx
from networkx.algorithms import bipartite
import json
import yaml
import ast
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
import pickle

import tensorflow as tf



In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=c41c9d58badfee65ac458db3a05f78f670b4c60318e3a38fed99ba06680c82ff
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Dmoz_Computers

In [ ]:
dataset_name = "Dmoz_Computers"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,Intelligence_1434272.txt,Principal Curves Page Introduction to principa...,Artificial,"[(principal curves, 0.7236), (curves summary, ...","[(introduction principal curves, 0.8212), (pri...","[(introduction principal curves, 0.8212), (pri...","[-0.034250993, -0.054697827, 0.02771881, -0.00...","[[-0.035950914, -0.084732965, -0.01326547, -0....","[[-0.068310104, -0.07682108, -0.0070457095, -0...","[[-0.068310104, -0.07682108, -0.0070457095, -0...","[[-0.034250993, -0.054697827, 0.02771881, -0.0..."
1,1,Intelligence_1433615.txt,Iowa State University - Artificial Intelligenc...,Artificial,"[(university artificial, 0.5531), (intelligenc...","[(university artificial intelligence, 0.6874),...","[(university artificial intelligence, 0.6874),...","[-0.037128627, -0.054665048, -0.023520134, -0....","[[0.013373093, -0.042429943, 0.0018845485, -0....","[[-0.03555484, -0.020074962, 0.039645746, -0.0...","[[-0.03555484, -0.020074962, 0.039645746, -0.0...","[[-0.037128627, -0.054665048, -0.023520134, -0..."
2,2,Intelligence_1434400.txt,"Paccanaro, Alberto Learning distributed repres...",Artificial,"[(concepts relational, 0.5298), (distributed r...","[(distributed representation concepts, 0.5994)...","[(distributed representation concepts, 0.5994)...","[0.017933754, -0.093935154, -0.07233604, 0.049...","[[0.09489803, -0.010703894, -0.07612236, 0.007...","[[-0.055929314, -0.041734546, -0.06326631, -0....","[[-0.055929314, -0.041734546, -0.06326631, -0....","[[0.017933754, -0.093935154, -0.07233604, 0.04..."
3,3,Intelligence_1434281.txt,Attrasoft Provider of a number of neural netwo...,Artificial,"[(sound recognition, 0.5352), (attrasoft provi...","[(image sound recognition, 0.5388), (neural ne...","[(image sound recognition, 0.5388), (sound rec...","[-0.09700832, -0.123844825, -0.025894646, 0.02...","[[-0.043651912, -0.09779582, -0.024362028, -0....","[[-0.02410375, -0.06722981, -0.016101753, -0.0...","[[-0.02410375, -0.06722981, -0.016101753, -0.0...","[[-0.09700832, -0.123844825, -0.025894646, 0.0..."
4,4,Intelligence_1434141.txt,LibML A machine learning library. New implemen...,Artificial,"[(machine learning, 0.614), (libml machine, 0....","[(libml machine learning, 0.8631), (machine le...","[(libml machine learning, 0.8631), (machine le...","[-0.08543352, -0.08932488, -0.019404357, -0.04...","[[-0.024391863, 0.0032444268, 0.054267664, -0....","[[-0.06194022, -0.10829034, 0.034314997, -0.02...","[[-0.06194022, -0.10829034, 0.034314997, -0.02...","[[-0.064178735, -0.12423259, -0.01962825, -0.0..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

9500


In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,Intelligence_1434272.txt,Principal Curves Page Introduction to principa...,Artificial,"[(principal curves, 0.7236), (curves summary, ...","[(introduction principal curves, 0.8212), (pri...","[(introduction principal curves, 0.8212), (pri...","[-0.034250993, -0.054697827, 0.02771881, -0.00...","[[-0.035950914, -0.084732965, -0.01326547, -0....","[[-0.068310104, -0.07682108, -0.0070457095, -0...","[[-0.068310104, -0.07682108, -0.0070457095, -0...","[[-0.034250993, -0.054697827, 0.02771881, -0.0..."
1,1,Intelligence_1433615.txt,Iowa State University - Artificial Intelligenc...,Artificial,"[(university artificial, 0.5531), (intelligenc...","[(university artificial intelligence, 0.6874),...","[(university artificial intelligence, 0.6874),...","[-0.037128627, -0.054665048, -0.023520134, -0....","[[0.013373093, -0.042429943, 0.0018845485, -0....","[[-0.03555484, -0.020074962, 0.039645746, -0.0...","[[-0.03555484, -0.020074962, 0.039645746, -0.0...","[[-0.037128627, -0.054665048, -0.023520134, -0..."
2,2,Intelligence_1434400.txt,"Paccanaro, Alberto Learning distributed repres...",Artificial,"[(concepts relational, 0.5298), (distributed r...","[(distributed representation concepts, 0.5994)...","[(distributed representation concepts, 0.5994)...","[0.017933754, -0.093935154, -0.07233604, 0.049...","[[0.09489803, -0.010703894, -0.07612236, 0.007...","[[-0.055929314, -0.041734546, -0.06326631, -0....","[[-0.055929314, -0.041734546, -0.06326631, -0....","[[0.017933754, -0.093935154, -0.07233604, 0.04..."
3,3,Intelligence_1434281.txt,Attrasoft Provider of a number of neural netwo...,Artificial,"[(sound recognition, 0.5352), (attrasoft provi...","[(image sound recognition, 0.5388), (neural ne...","[(image sound recognition, 0.5388), (sound rec...","[-0.09700832, -0.123844825, -0.025894646, 0.02...","[[-0.043651912, -0.09779582, -0.024362028, -0....","[[-0.02410375, -0.06722981, -0.016101753, -0.0...","[[-0.02410375, -0.06722981, -0.016101753, -0.0...","[[-0.09700832, -0.123844825, -0.025894646, 0.0..."
4,4,Intelligence_1434141.txt,LibML A machine learning library. New implemen...,Artificial,"[(machine learning, 0.614), (libml machine, 0....","[(libml machine learning, 0.8631), (machine le...","[(libml machine learning, 0.8631), (machine le...","[-0.08543352, -0.08932488, -0.019404357, -0.04...","[[-0.024391863, 0.0032444268, 0.054267664, -0....","[[-0.06194022, -0.10829034, 0.034314997, -0.02...","[[-0.06194022, -0.10829034, 0.034314997, -0.02...","[[-0.064178735, -0.12423259, -0.01962825, -0.0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# Dmoz_Health

In [ ]:
dataset_name = "Dmoz_Health"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,1578510.txt,Illinois Church Action on Alcohol and Addictio...,Addictions,"[(illinois church, 0.6463), (groups illinois, ...","[(illinois church action, 0.7092), (church act...","[(illinois church action, 0.7092), (illinois c...","[0.02022246, 0.022007184, 0.0019790446, -0.001...","[[-0.013755142, 0.024827007, 0.010499118, -0.0...","[[-0.027233787, 0.04967187, 0.018105166, -0.05...","[[-0.027233787, 0.04967187, 0.018105166, -0.05..."
1,1,1577747.txt,AA Statewide Meeting lists for all of Vermont....,Addictions,"[(alcoholics anonymous, 0.6022), (lists vermon...","[(meeting lists vermont, 0.7619), (aa statewid...","[(meeting lists vermont, 0.7619), (aa statewid...","[0.03485902, -0.021918835, 0.0015977684, 0.025...","[[-0.063851565, 0.02354454, -0.057326257, 0.04...","[[0.017069533, -0.032487243, 0.04040509, 0.027...","[[0.017069533, -0.032487243, 0.04040509, 0.027..."
2,2,1578166.txt,Gracer Medical Group Dr. Richard Gracer is a p...,Addictions,"[(richard gracer, 0.7271), (gracer medical, 0....","[(dr richard gracer, 0.7878), (richard gracer ...","[(dr richard gracer, 0.7878), (richard gracer ...","[-0.006266652, 0.059151437, 0.009448245, 0.011...","[[-0.06466706, 0.06959263, -0.010909742, -0.03...","[[-0.06400401, 0.08335317, 0.0007347387, -0.01...","[[-0.06400401, 0.08335317, 0.0007347387, -0.01..."
3,3,1577381.txt,"Phoenix Meetings, events, and visitor informat...",Addictions,"[(phoenix meetings, 0.5868), (mesa phoenix, 0....","[(salt river intergroup, 0.6306), (phoenix mee...","[(salt river intergroup, 0.6306), (phoenix mee...","[0.06477824, -0.048163723, -0.032376703, -0.01...","[[0.041145816, -0.0019022514, -0.08633919, 0.0...","[[-0.03362186, -0.014215576, 0.02653337, -0.01...","[[-0.03362186, -0.014215576, 0.02653337, -0.01..."
4,4,1578793.txt,American River Area Narcotics Anonymous Resour...,Addictions,"[(narcotics anonymous, 0.5901), (addiction sac...","[(area narcotics anonymous, 0.6939), (narcotic...","[(area narcotics anonymous, 0.6939), (narcotic...","[0.028619595, -0.01218137, -0.0069289766, 0.03...","[[-0.08115753, 0.045479596, -0.057911057, 0.00...","[[-0.034894887, 0.017864896, -0.040609572, 0.0...","[[-0.034894887, 0.017864896, -0.040609572, 0.0..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,1578510.txt,Illinois Church Action on Alcohol and Addictio...,Addictions,"[(illinois church, 0.6463), (groups illinois, ...","[(illinois church action, 0.7092), (church act...","[(illinois church action, 0.7092), (illinois c...","[0.02022246, 0.022007184, 0.0019790446, -0.001...","[[-0.013755142, 0.024827007, 0.010499118, -0.0...","[[-0.027233787, 0.04967187, 0.018105166, -0.05...","[[-0.027233787, 0.04967187, 0.018105166, -0.05...","[[0.02022246, 0.022007184, 0.0019790446, -0.00..."
1,1,1577747.txt,AA Statewide Meeting lists for all of Vermont....,Addictions,"[(alcoholics anonymous, 0.6022), (lists vermon...","[(meeting lists vermont, 0.7619), (aa statewid...","[(meeting lists vermont, 0.7619), (aa statewid...","[0.03485902, -0.021918835, 0.0015977684, 0.025...","[[-0.063851565, 0.02354454, -0.057326257, 0.04...","[[0.017069533, -0.032487243, 0.04040509, 0.027...","[[0.017069533, -0.032487243, 0.04040509, 0.027...","[[0.030876612, -0.033414394, 0.03301465, 0.009..."
2,2,1578166.txt,Gracer Medical Group Dr. Richard Gracer is a p...,Addictions,"[(richard gracer, 0.7271), (gracer medical, 0....","[(dr richard gracer, 0.7878), (richard gracer ...","[(dr richard gracer, 0.7878), (richard gracer ...","[-0.006266652, 0.059151437, 0.009448245, 0.011...","[[-0.06466706, 0.06959263, -0.010909742, -0.03...","[[-0.06400401, 0.08335317, 0.0007347387, -0.01...","[[-0.06400401, 0.08335317, 0.0007347387, -0.01...","[[-0.006266652, 0.059151437, 0.009448245, 0.01..."
3,3,1577381.txt,"Phoenix Meetings, events, and visitor informat...",Addictions,"[(phoenix meetings, 0.5868), (mesa phoenix, 0....","[(salt river intergroup, 0.6306), (phoenix mee...","[(salt river intergroup, 0.6306), (phoenix mee...","[0.06477824, -0.048163723, -0.032376703, -0.01...","[[0.041145816, -0.0019022514, -0.08633919, 0.0...","[[-0.03362186, -0.014215576, 0.02653337, -0.01...","[[-0.03362186, -0.014215576, 0.02653337, -0.01...","[[0.060021464, 0.007135069, 0.0019532838, 0.04..."
4,4,1578793.txt,American River Area Narcotics Anonymous Resour...,Addictions,"[(narcotics anonymous, 0.5901), (addiction sac...","[(area narcotics anonymous, 0.6939), (narcotic...","[(area narcotics anonymous, 0.6939), (narcotic...","[0.028619595, -0.01218137, -0.0069289766, 0.03...","[[-0.08115753, 0.045479596, -0.057911057, 0.00...","[[-0.034894887, 0.017864896, -0.040609572, 0.0...","[[-0.034894887, 0.017864896, -0.040609572, 0.0...","[[0.022866843, -0.009676686, -0.027952429, 0.0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# CSTR

In [ ]:
dataset_name = "CSTR"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,126.txt,Rhetorical (Rhet) is a programming / knowledge...,ArtificiallIntelligence,"[(rhetorical rhet, 0.7475), (rhet programming,...","[(rhetorical rhet programming, 0.8299), (rhet ...","[(rhetorical rhet programming, 0.8299), (rheto...","[-0.022039127, 0.02312275, -0.076888025, -0.06...","[[0.0059065777, 0.040003918, 0.0011322909, -0....","[[-0.045958184, 0.03794197, -0.027850023, -0.0...","[[-0.045958184, 0.03794197, -0.027850023, -0.0..."
1,1,5.txt,Reduction is the operation of transforming a p...,ArtificiallIntelligence,"[(parsing complexity, 0.5977), (transformed gr...","[(complexity resulting grammar, 0.675), (minim...","[(complexity resulting grammar, 0.675), (minim...","[0.074621424, -0.007094671, 0.051812325, -0.03...","[[0.0426418, -0.013719074, 0.02755835, -0.0075...","[[0.07436234, -0.0029446871, 0.052716985, 0.03...","[[0.07436234, -0.0029446871, 0.052716985, 0.03..."
2,2,48.txt,"For years, researchers have used knowledge-int...",ArtificiallIntelligence,"[(disambiguating parsing, 0.7122), (syntactic ...","[(techniques disambiguating parsing, 0.7356), ...","[(techniques disambiguating parsing, 0.7356), ...","[-0.014407578, -0.077485226, 0.083884224, 0.01...","[[0.014474175, -0.027266933, 0.07052159, -0.02...","[[0.022918042, -0.01977642, 0.047498055, -0.01...","[[0.022918042, -0.01977642, 0.047498055, -0.01..."
3,3,81.txt,Proceedings of a workshop held in conjunction ...,ArtificiallIntelligence,"[(machine learning, 0.6059), (tahoe city, 0.39...","[(machine learning july, 0.663), (conference m...","[(machine learning july, 0.663), (machine lear...","[-0.036386, -0.08516384, 0.06518213, 0.0014036...","[[-0.024391863, 0.0032444268, 0.054267664, -0....","[[-0.075079046, -0.10737129, 0.06574192, 0.002...","[[-0.075079046, -0.10737129, 0.06574192, 0.002..."
4,4,25.txt,The Medication Advisor is the latest project o...,ArtificiallIntelligence,"[(medication advisor, 0.5975), (conversational...","[(medications medication advisor, 0.62), (cons...","[(medications medication advisor, 0.62), (cons...","[-0.050106738, -0.013832041, 0.03398316, -0.02...","[[-0.030672388, 0.007967623, -0.023997275, 0.0...","[[-0.020701267, 0.003092026, -0.024871623, 0.0...","[[-0.020701267, 0.003092026, -0.024871623, 0.0..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,126.txt,Rhetorical (Rhet) is a programming / knowledge...,ArtificiallIntelligence,"[(rhetorical rhet, 0.7475), (rhet programming,...","[(rhetorical rhet programming, 0.8299), (rhet ...","[(rhetorical rhet programming, 0.8299), (rheto...","[-0.022039127, 0.02312275, -0.076888025, -0.06...","[[0.0059065777, 0.040003918, 0.0011322909, -0....","[[-0.045958184, 0.03794197, -0.027850023, -0.0...","[[-0.045958184, 0.03794197, -0.027850023, -0.0...","[[0.0026859022, 0.0066884025, -0.07187084, -0...."
1,1,5.txt,Reduction is the operation of transforming a p...,ArtificiallIntelligence,"[(parsing complexity, 0.5977), (transformed gr...","[(complexity resulting grammar, 0.675), (minim...","[(complexity resulting grammar, 0.675), (minim...","[0.074621424, -0.007094671, 0.051812325, -0.03...","[[0.0426418, -0.013719074, 0.02755835, -0.0075...","[[0.07436234, -0.0029446871, 0.052716985, 0.03...","[[0.07436234, -0.0029446871, 0.052716985, 0.03...","[[0.008135828, 0.0732408, 0.030002978, -0.0219..."
2,2,48.txt,"For years, researchers have used knowledge-int...",ArtificiallIntelligence,"[(disambiguating parsing, 0.7122), (syntactic ...","[(techniques disambiguating parsing, 0.7356), ...","[(techniques disambiguating parsing, 0.7356), ...","[-0.014407578, -0.077485226, 0.083884224, 0.01...","[[0.014474175, -0.027266933, 0.07052159, -0.02...","[[0.022918042, -0.01977642, 0.047498055, -0.01...","[[0.022918042, -0.01977642, 0.047498055, -0.01...","[[0.013398974, -0.025847042, 0.0290076, 0.0357..."
3,3,81.txt,Proceedings of a workshop held in conjunction ...,ArtificiallIntelligence,"[(machine learning, 0.6059), (tahoe city, 0.39...","[(machine learning july, 0.663), (conference m...","[(machine learning july, 0.663), (machine lear...","[-0.036386, -0.08516384, 0.06518213, 0.0014036...","[[-0.024391863, 0.0032444268, 0.054267664, -0....","[[-0.075079046, -0.10737129, 0.06574192, 0.002...","[[-0.075079046, -0.10737129, 0.06574192, 0.002...","[[-0.036386, -0.08516384, 0.06518213, 0.001403..."
4,4,25.txt,The Medication Advisor is the latest project o...,ArtificiallIntelligence,"[(medication advisor, 0.5975), (conversational...","[(medications medication advisor, 0.62), (cons...","[(medications medication advisor, 0.62), (cons...","[-0.050106738, -0.013832041, 0.03398316, -0.02...","[[-0.030672388, 0.007967623, -0.023997275, 0.0...","[[-0.020701267, 0.003092026, -0.024871623, 0.0...","[[-0.020701267, 0.003092026, -0.024871623, 0.0...","[[-0.05715183, -0.03530269, 0.015551494, -0.00..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# Industry_Sector

In [ ]:
dataset_name = "Industry_Sector"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,materials_http:^^www.littonacdipe.com,Litton Litton: Interconnect Prod...,basic,"[(packaging esp, 0.5861), (litton interconnect...","[(electronic systems packaging, 0.6613), (syst...","[(electronic systems packaging, 0.6613), (syst...","[-0.05860152, 0.0145570105, 0.017551647, -0.02...","[[-0.07081439, 0.053046886, 0.038261086, -0.04...","[[-0.07142605, 0.085836, -0.04343102, -0.05472...","[[-0.07142605, 0.085836, -0.04343102, -0.05472..."
1,1,materials_http:^^www.weirton.com^contact.htm,"Allow: GET, HEAD URI: Contact Weirton St...",basic,"[(contact firm, 0.562), (contact weirton, 0.50...","[(contact weirton steel, 0.5781), (come contac...","[(contact weirton steel, 0.5781), (contact fir...","[-0.124327414, -0.012992704, 0.014001042, 0.03...","[[-0.08809786, -0.019295853, 0.00693113, -0.01...","[[-0.10939312, -0.018659644, 0.051947754, 0.07...","[[-0.10939312, -0.018659644, 0.051947754, 0.07..."
2,2,materials_http:^^home.earthlink.net^~nuco2^Bul...,Systems Which system would you choose?...,basic,"[(pressure cylinders, 0.6841), (pressure bulk,...","[(high pressure cylinders, 0.7351), (choose lo...","[(high pressure cylinders, 0.7351), (pressure ...","[-0.085865036, 0.043530438, -0.071711995, -0.0...","[[-0.097670145, 0.07834313, -0.067915626, -0.0...","[[-0.093530454, 0.07404681, -0.049339183, -0.0...","[[-0.093530454, 0.07404681, -0.049339183, -0.0..."
3,3,materials_http:^^www.useg.com^invest.html,US Energy Corporation Investor News ...,basic,"[(images investon, 0.5688), (gif investon, 0.5...","[(images investon gif, 0.6559), (images invest...","[(images investon gif, 0.6559), (images invest...","[0.008732203, -0.013307741, -0.036661223, 0.02...","[[-0.009558729, 0.017956274, -0.059580665, -0....","[[0.00017329463, 0.0013028142, -0.06374502, -0...","[[0.00017329463, 0.0013028142, -0.06374502, -0..."
4,4,materials_http:^^www.terraindustries.com^about...,"Terra Industries, Inc., Company Informat...",basic,"[(terra nitrogen, 0.6228), (nitrogen fertilize...","[(terra produces nitrogen, 0.7287), (terra nit...","[(terra produces nitrogen, 0.7287), (terra nit...","[-0.049107973, -0.043558124, -0.020951217, -0....","[[-0.07773064, 0.033809956, 0.025796082, -0.01...","[[-0.045650914, 0.045816682, 0.023411866, -0.0...","[[-0.045650914, 0.045816682, 0.023411866, -0.0..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,materials_http:^^www.littonacdipe.com,Litton Litton: Interconnect Prod...,basic,"[(packaging esp, 0.5861), (litton interconnect...","[(electronic systems packaging, 0.6613), (syst...","[(electronic systems packaging, 0.6613), (syst...","[-0.05860152, 0.0145570105, 0.017551647, -0.02...","[[-0.07081439, 0.053046886, 0.038261086, -0.04...","[[-0.07142605, 0.085836, -0.04343102, -0.05472...","[[-0.07142605, 0.085836, -0.04343102, -0.05472...","[[-0.0956507, -0.0512441, -0.009371761, 0.0161..."
1,1,materials_http:^^www.weirton.com^contact.htm,"Allow: GET, HEAD URI: Contact Weirton St...",basic,"[(contact firm, 0.562), (contact weirton, 0.50...","[(contact weirton steel, 0.5781), (come contac...","[(contact weirton steel, 0.5781), (contact fir...","[-0.124327414, -0.012992704, 0.014001042, 0.03...","[[-0.08809786, -0.019295853, 0.00693113, -0.01...","[[-0.10939312, -0.018659644, 0.051947754, 0.07...","[[-0.10939312, -0.018659644, 0.051947754, 0.07...","[[-0.09154996, 0.004012895, 0.07021415, -0.001..."
2,2,materials_http:^^home.earthlink.net^~nuco2^Bul...,Systems Which system would you choose?...,basic,"[(pressure cylinders, 0.6841), (pressure bulk,...","[(high pressure cylinders, 0.7351), (choose lo...","[(high pressure cylinders, 0.7351), (pressure ...","[-0.085865036, 0.043530438, -0.071711995, -0.0...","[[-0.097670145, 0.07834313, -0.067915626, -0.0...","[[-0.093530454, 0.07404681, -0.049339183, -0.0...","[[-0.093530454, 0.07404681, -0.049339183, -0.0...","[[-0.03351223, -0.006459103, -0.07732453, -0.0..."
3,3,materials_http:^^www.useg.com^invest.html,US Energy Corporation Investor News ...,basic,"[(images investon, 0.5688), (gif investon, 0.5...","[(images investon gif, 0.6559), (images invest...","[(images investon gif, 0.6559), (images invest...","[0.008732203, -0.013307741, -0.036661223, 0.02...","[[-0.009558729, 0.017956274, -0.059580665, -0....","[[0.00017329463, 0.0013028142, -0.06374502, -0...","[[0.00017329463, 0.0013028142, -0.06374502, -0...","[[0.008732203, -0.013307741, -0.036661223, 0.0..."
4,4,materials_http:^^www.terraindustries.com^about...,"Terra Industries, Inc., Company Informat...",basic,"[(terra nitrogen, 0.6228), (nitrogen fertilize...","[(terra produces nitrogen, 0.7287), (terra nit...","[(terra produces nitrogen, 0.7287), (terra nit...","[-0.049107973, -0.043558124, -0.020951217, -0....","[[-0.07773064, 0.033809956, 0.025796082, -0.01...","[[-0.045650914, 0.045816682, 0.023411866, -0.0...","[[-0.045650914, 0.045816682, 0.023411866, -0.0...","[[-0.034066442, 0.033516087, 0.018464932, -0.0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# re8

In [ ]:
dataset_name = "re8"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,acq.4342.txt,mcdowell me to merger with interpharm inc mcdo...,acq,"[(pct interpharm, 0.6568), (interpharm pct, 0....","[(mcdowell merger interpharm, 0.7512), (acquir...","[(mcdowell merger interpharm, 0.7512), (acquir...","[-0.027460506, -0.044152174, 0.094381206, -0.0...","[[-0.10132556, -0.037788533, 0.06360753, -0.03...","[[-0.054724135, -0.02879868, 0.07816885, -0.05...","[[-0.054724135, -0.02879868, 0.07816885, -0.05..."
1,1,acq.5302.txt,intermagnetics general inma completes buy inte...,acq,"[(buy intermagnetics, 0.5958), (intermagnetics...","[(completes buy intermagnetics, 0.6764), (inte...","[(completes buy intermagnetics, 0.6764), (inte...","[-0.123292044, -0.016547788, 0.055199977, -0.0...","[[-0.11072712, -0.012237637, -0.014712719, -0....","[[-0.12598571, 0.00096617965, 0.0040173163, -0...","[[-0.12598571, 0.00096617965, 0.0040173163, -0..."
2,2,acq.8530.txt,tesco extends hillards offer tesco plc tsco l ...,acq,"[(tesco plc, 0.5533), (offer tesco, 0.5289), (...","[(hillards offer tesco, 0.6338), (pct hillards...","[(hillards offer tesco, 0.6338), (pct hillards...","[-0.04780644, -0.03977985, 0.0098251505, -0.01...","[[-0.0544593, 0.018437698, -0.03501143, -0.026...","[[0.0075517837, 0.024240255, -0.031178677, -0....","[[0.0075517837, 0.024240255, -0.031178677, -0...."
3,3,acq.3841.txt,healthvest hvt sells shares healthvest a maryl...,acq,"[(shares healthvest, 0.6659), (hvt sells, 0.63...","[(hvt sells shares, 0.7235), (healthvest hvt s...","[(hvt sells shares, 0.7235), (healthvest hvt s...","[-0.018709125, -0.02001635, 0.02927253, -0.046...","[[0.033341378, 0.03394121, -0.04865912, 0.0476...","[[0.0012255874, -0.016721591, -0.005323345, -0...","[[0.0012255874, -0.016721591, -0.005323345, -0..."
4,4,acq.6302.txt,cooper canada said it received takeover offers...,acq,"[(takeover offers, 0.6083), (received takeover...","[(takeover offers cooper, 0.7835), (received t...","[(takeover offers cooper, 0.7835), (received t...","[-0.038452294, -0.043767497, 0.07514506, 0.008...","[[-0.04487832, -0.009658501, 0.056613285, -0.0...","[[-0.0373211, 0.008434889, 0.034872495, 0.0356...","[[-0.0373211, 0.008434889, 0.034872495, 0.0356..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,acq.4342.txt,mcdowell me to merger with interpharm inc mcdo...,acq,"[(pct interpharm, 0.6568), (interpharm pct, 0....","[(mcdowell merger interpharm, 0.7512), (acquir...","[(mcdowell merger interpharm, 0.7512), (acquir...","[-0.027460506, -0.044152174, 0.094381206, -0.0...","[[-0.10132556, -0.037788533, 0.06360753, -0.03...","[[-0.054724135, -0.02879868, 0.07816885, -0.05...","[[-0.054724135, -0.02879868, 0.07816885, -0.05...","[[-0.027460506, -0.044152174, 0.094381206, -0...."
1,1,acq.5302.txt,intermagnetics general inma completes buy inte...,acq,"[(buy intermagnetics, 0.5958), (intermagnetics...","[(completes buy intermagnetics, 0.6764), (inte...","[(completes buy intermagnetics, 0.6764), (inte...","[-0.123292044, -0.016547788, 0.055199977, -0.0...","[[-0.11072712, -0.012237637, -0.014712719, -0....","[[-0.12598571, 0.00096617965, 0.0040173163, -0...","[[-0.12598571, 0.00096617965, 0.0040173163, -0...","[[-0.123292044, -0.016547788, 0.055199977, -0...."
2,2,acq.8530.txt,tesco extends hillards offer tesco plc tsco l ...,acq,"[(tesco plc, 0.5533), (offer tesco, 0.5289), (...","[(hillards offer tesco, 0.6338), (pct hillards...","[(hillards offer tesco, 0.6338), (pct hillards...","[-0.04780644, -0.03977985, 0.0098251505, -0.01...","[[-0.0544593, 0.018437698, -0.03501143, -0.026...","[[0.0075517837, 0.024240255, -0.031178677, -0....","[[0.0075517837, 0.024240255, -0.031178677, -0....","[[-0.04780644, -0.03977985, 0.0098251505, -0.0..."
3,3,acq.3841.txt,healthvest hvt sells shares healthvest a maryl...,acq,"[(shares healthvest, 0.6659), (hvt sells, 0.63...","[(hvt sells shares, 0.7235), (healthvest hvt s...","[(hvt sells shares, 0.7235), (healthvest hvt s...","[-0.018709125, -0.02001635, 0.02927253, -0.046...","[[0.033341378, 0.03394121, -0.04865912, 0.0476...","[[0.0012255874, -0.016721591, -0.005323345, -0...","[[0.0012255874, -0.016721591, -0.005323345, -0...","[[-0.018709125, -0.02001635, 0.02927253, -0.04..."
4,4,acq.6302.txt,cooper canada said it received takeover offers...,acq,"[(takeover offers, 0.6083), (received takeover...","[(takeover offers cooper, 0.7835), (received t...","[(takeover offers cooper, 0.7835), (received t...","[-0.038452294, -0.043767497, 0.07514506, 0.008...","[[-0.04487832, -0.009658501, 0.056613285, -0.0...","[[-0.0373211, 0.008434889, 0.034872495, 0.0356...","[[-0.0373211, 0.008434889, 0.034872495, 0.0356...","[[-0.038452294, -0.043767497, 0.07514506, 0.00..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# NSF

In [ ]:
dataset_name = "NSF"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,management_data_management.a9820721.txt,an intelligent visual database system hierarch...,data,"[(description matching, 0.5734), (integrated s...","[(visual database hierarchical, 0.6701), (inte...","[(visual database hierarchical, 0.6701), (inte...","[0.047953185, -0.055671744, 0.005448584, -0.04...","[[-0.05046647, 0.11629063, 0.063316606, -0.038...","[[0.05310088, -0.09617194, 0.021846347, 0.0368...","[[0.05310088, -0.09617194, 0.021846347, 0.0368..."
1,1,management_data_management.a9116988.txt,spatio-temporal database management for global...,data,"[(temporal database, 0.6222), (database manage...","[(temporal database management, 0.7363), (spat...","[(temporal database management, 0.7363), (spat...","[0.015489308, -0.07953871, 0.016742839, -0.018...","[[0.015882665, -0.0050327675, -0.020824937, -0...","[[-0.005868738, 0.009880877, -0.044446137, -0....","[[-0.005868738, 0.009880877, -0.044446137, -0...."
2,2,management_data_management.a9457613.txt,nyi theory and implementation of declarative d...,data,"[(declarative database, 0.715), (database syst...","[(declarative database systems, 0.8109), (impl...","[(declarative database systems, 0.8109), (impl...","[-0.0026100776, -0.000591449, -0.016258776, -0...","[[0.049995393, -0.020903759, -0.028571002, -0....","[[0.007936481, -0.0021183742, -0.023153529, -0...","[[0.007936481, -0.0021183742, -0.023153529, -0..."
3,3,management_data_management.a9734191.txt,pecase providing a coherent view of diverse di...,data,"[(pecase providing, 0.5841), (diverse distribu...","[(distributed information resources, 0.7096), ...","[(distributed information resources, 0.7096), ...","[-0.021243388, -0.028308352, -0.017791938, -0....","[[-0.032006714, 0.065156184, 0.049498767, -0.1...","[[-0.02908297, -0.032851376, -0.035267286, 0.0...","[[-0.02908297, -0.032851376, -0.035267286, 0.0..."
4,4,management_data_management.a9116798.txt,an object-oriented toolbox for use with the pr...,data,"[(protein data, 0.6531), (use protein, 0.4776)...","[(protein data bank, 0.7741), (use protein dat...","[(protein data bank, 0.7741), (use protein dat...","[-0.04303045, -0.042519566, -0.07707622, -0.07...","[[-0.0091545805, -0.06159268, -0.04229239, -0....","[[-0.04777056, -0.07305523, -0.05686991, -0.04...","[[-0.04777056, -0.07305523, -0.05686991, -0.04..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,management_data_management.a9820721.txt,an intelligent visual database system hierarch...,data,"[(description matching, 0.5734), (integrated s...","[(visual database hierarchical, 0.6701), (inte...","[(visual database hierarchical, 0.6701), (inte...","[0.047953185, -0.055671744, 0.005448584, -0.04...","[[-0.05046647, 0.11629063, 0.063316606, -0.038...","[[0.05310088, -0.09617194, 0.021846347, 0.0368...","[[0.05310088, -0.09617194, 0.021846347, 0.0368...","[[0.047953185, -0.055671744, 0.005448584, -0.0..."
1,1,management_data_management.a9116988.txt,spatio-temporal database management for global...,data,"[(temporal database, 0.6222), (database manage...","[(temporal database management, 0.7363), (spat...","[(temporal database management, 0.7363), (spat...","[0.015489308, -0.07953871, 0.016742839, -0.018...","[[0.015882665, -0.0050327675, -0.020824937, -0...","[[-0.005868738, 0.009880877, -0.044446137, -0....","[[-0.005868738, 0.009880877, -0.044446137, -0....","[[0.015489308, -0.07953871, 0.016742839, -0.01..."
2,2,management_data_management.a9457613.txt,nyi theory and implementation of declarative d...,data,"[(declarative database, 0.715), (database syst...","[(declarative database systems, 0.8109), (impl...","[(declarative database systems, 0.8109), (impl...","[-0.0026100776, -0.000591449, -0.016258776, -0...","[[0.049995393, -0.020903759, -0.028571002, -0....","[[0.007936481, -0.0021183742, -0.023153529, -0...","[[0.007936481, -0.0021183742, -0.023153529, -0...","[[-0.0026100776, -0.000591449, -0.016258776, -..."
3,3,management_data_management.a9734191.txt,pecase providing a coherent view of diverse di...,data,"[(pecase providing, 0.5841), (diverse distribu...","[(distributed information resources, 0.7096), ...","[(distributed information resources, 0.7096), ...","[-0.021243388, -0.028308352, -0.017791938, -0....","[[-0.032006714, 0.065156184, 0.049498767, -0.1...","[[-0.02908297, -0.032851376, -0.035267286, 0.0...","[[-0.02908297, -0.032851376, -0.035267286, 0.0...","[[-0.021243388, -0.028308352, -0.017791938, -0..."
4,4,management_data_management.a9116798.txt,an object-oriented toolbox for use with the pr...,data,"[(protein data, 0.6531), (use protein, 0.4776)...","[(protein data bank, 0.7741), (use protein dat...","[(protein data bank, 0.7741), (use protein dat...","[-0.04303045, -0.042519566, -0.07707622, -0.07...","[[-0.0091545805, -0.06159268, -0.04229239, -0....","[[-0.04777056, -0.07305523, -0.05686991, -0.04...","[[-0.04777056, -0.07305523, -0.05686991, -0.04...","[[-0.04303045, -0.042519566, -0.07707622, -0.0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# Dmoz_Sports

In [ ]:
dataset_name = "Dmoz_Sports"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,3191005.txt,"Bryant Bulldogs Official site with news, sched...",Baseball,"[(bryant bulldogs, 0.8054), (bulldogs official...","[(bryant bulldogs official, 0.8295), (bulldogs...","[(bryant bulldogs official, 0.8295), (bryant b...","[-0.055418193, -0.04596237, -0.04405278, 0.041...","[[-0.072696306, 0.0038710712, -0.040226072, 0....","[[-0.09256799, 0.012199265, -0.043049898, -0.0...","[[-0.09256799, 0.012199265, -0.043049898, -0.0..."
1,1,3193010.txt,Hirschbeck v. Alomar Covers a legal case betwe...,Baseball,"[(legal case, 0.5041), (hirschbeck alomar, 0.4...","[(case player umpire, 0.5295), (legal case pla...","[(case player umpire, 0.5295), (legal case pla...","[-0.05894342, 0.112123415, -0.029518068, -0.06...","[[-0.010286466, 0.07685405, -0.0046042153, -0....","[[0.047361054, 0.11441055, -0.045774344, -0.10...","[[0.047361054, 0.11441055, -0.045774344, -0.10..."
2,2,3190744.txt,Mountaineers Baseball Official site. Includes ...,Baseball,"[(mountaineers baseball, 0.7208), (official si...","[(mountaineers baseball official, 0.8164), (ba...","[(mountaineers baseball official, 0.8164), (mo...","[-0.0773655, 0.0049256724, 0.0047015045, -0.01...","[[-0.11355926, 0.07837829, -0.006943615, -0.03...","[[-0.117959335, 0.06918415, 0.0020498591, -0.0...","[[-0.117959335, 0.06918415, 0.0020498591, -0.0..."
3,3,3193695.txt,"M.S.H. Little League Contains news items, a li...",Baseball,"[(little league, 0.6328), (listing teams, 0.50...","[(little league contains, 0.6704), (league con...","[(little league contains, 0.6704), (little lea...","[0.064685404, 0.077866, -0.04203705, 0.0274158...","[[0.0437485, 0.09528501, -0.06670673, 0.001467...","[[0.08467223, 0.03760568, -0.028927764, -0.011...","[[0.08467223, 0.03760568, -0.028927764, -0.011..."
4,4,3193486.txt,Quotations: Bob Uecker Baseball Almanac collec...,Baseball,"[(quotes ballplayer, 0.6472), (quotations bob,...","[(quotations bob uecker, 0.6792), (quotes ball...","[(quotations bob uecker, 0.6792), (quotes ball...","[-0.062535815, 0.08351713, -0.015831418, -0.02...","[[-0.03655811, 0.030010985, 0.020585416, -0.03...","[[-0.072881274, 0.06380257, 0.015473793, 0.026...","[[-0.072881274, 0.06380257, 0.015473793, 0.026..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,3191005.txt,"Bryant Bulldogs Official site with news, sched...",Baseball,"[(bryant bulldogs, 0.8054), (bulldogs official...","[(bryant bulldogs official, 0.8295), (bulldogs...","[(bryant bulldogs official, 0.8295), (bryant b...","[-0.055418193, -0.04596237, -0.04405278, 0.041...","[[-0.072696306, 0.0038710712, -0.040226072, 0....","[[-0.09256799, 0.012199265, -0.043049898, -0.0...","[[-0.09256799, 0.012199265, -0.043049898, -0.0...","[[-0.055418193, -0.04596237, -0.04405278, 0.04..."
1,1,3193010.txt,Hirschbeck v. Alomar Covers a legal case betwe...,Baseball,"[(legal case, 0.5041), (hirschbeck alomar, 0.4...","[(case player umpire, 0.5295), (legal case pla...","[(case player umpire, 0.5295), (legal case pla...","[-0.05894342, 0.112123415, -0.029518068, -0.06...","[[-0.010286466, 0.07685405, -0.0046042153, -0....","[[0.047361054, 0.11441055, -0.045774344, -0.10...","[[0.047361054, 0.11441055, -0.045774344, -0.10...","[[-0.05894342, 0.112123415, -0.029518068, -0.0..."
2,2,3190744.txt,Mountaineers Baseball Official site. Includes ...,Baseball,"[(mountaineers baseball, 0.7208), (official si...","[(mountaineers baseball official, 0.8164), (ba...","[(mountaineers baseball official, 0.8164), (mo...","[-0.0773655, 0.0049256724, 0.0047015045, -0.01...","[[-0.11355926, 0.07837829, -0.006943615, -0.03...","[[-0.117959335, 0.06918415, 0.0020498591, -0.0...","[[-0.117959335, 0.06918415, 0.0020498591, -0.0...","[[-0.096298136, 0.035977315, -0.021594549, -0...."
3,3,3193695.txt,"M.S.H. Little League Contains news items, a li...",Baseball,"[(little league, 0.6328), (listing teams, 0.50...","[(little league contains, 0.6704), (league con...","[(little league contains, 0.6704), (little lea...","[0.064685404, 0.077866, -0.04203705, 0.0274158...","[[0.0437485, 0.09528501, -0.06670673, 0.001467...","[[0.08467223, 0.03760568, -0.028927764, -0.011...","[[0.08467223, 0.03760568, -0.028927764, -0.011...","[[-0.053826824, 0.0072779665, 0.0051527363, 0...."
4,4,3193486.txt,Quotations: Bob Uecker Baseball Almanac collec...,Baseball,"[(quotes ballplayer, 0.6472), (quotations bob,...","[(quotations bob uecker, 0.6792), (quotes ball...","[(quotations bob uecker, 0.6792), (quotes ball...","[-0.062535815, 0.08351713, -0.015831418, -0.02...","[[-0.03655811, 0.030010985, 0.020585416, -0.03...","[[-0.072881274, 0.06380257, 0.015473793, 0.026...","[[-0.072881274, 0.06380257, 0.015473793, 0.026...","[[-0.062535815, 0.08351713, -0.015831418, -0.0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# Dmoz_Science

In [ ]:
dataset_name = "Dmoz_Science"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,2786497.txt,Texas A&amp;M Horticultural Extension - Vegeta...,Agriculture,"[(extension vegetables, 0.6118), (newsletters ...","[(vegetable grower handbook, 0.7008), (newslet...","[(vegetable grower handbook, 0.7008), (newslet...","[0.016607286, -0.03996504, 0.01082185, 0.05656...","[[-0.03316761, 0.004545155, -0.05236538, 0.022...","[[-0.079359375, -0.014674136, -0.07064848, -0....","[[-0.079359375, -0.014674136, -0.07064848, -0...."
1,1,2784741.txt,Algaculture Information from Wikipedia on this...,Agriculture,"[(production microalgae, 0.7194), (algaculture...","[(algaculture information wikipedia, 0.7318), ...","[(algaculture information wikipedia, 0.7318), ...","[-0.052518472, 0.07755526, -0.09522597, 0.0270...","[[-0.035396907, -0.001984206, -0.060108755, -0...","[[0.0007872191, 0.027178274, -0.08233266, 0.03...","[[0.0007872191, 0.027178274, -0.08233266, 0.03..."
2,2,2785016.txt,Annual Canarygrass Factsheet on this grain cro...,Agriculture,"[(annual canarygrass, 0.7755), (canarygrass fa...","[(annual canarygrass factsheet, 0.7986), (cana...","[(annual canarygrass factsheet, 0.7986), (annu...","[0.03832742, 0.018079218, -0.039859474, 0.0059...","[[0.013984093, 0.0032539438, 0.012409236, 0.00...","[[0.030801402, 0.013365954, -0.010543321, 0.01...","[[0.030801402, 0.013365954, -0.010543321, 0.01..."
3,3,2786133.txt,Black Sigatoka Photographs and information on ...,Agriculture,"[(disease bananas, 0.681), (fungal disease, 0....","[(fungal disease bananas, 0.8024), (disease ba...","[(fungal disease bananas, 0.8024), (disease ba...","[-0.0030358238, 0.058499087, -0.08342932, -0.0...","[[-0.034461573, 0.0020409657, -0.01293984, 0.0...","[[-0.018461134, 0.02032768, -0.049900673, -0.0...","[[-0.018461134, 0.02032768, -0.049900673, -0.0..."
4,4,2786989.txt,Irrigation Training and Research Center Univer...,Agriculture,"[(irrigation training, 0.8181), (focusing irri...","[(irrigation training research, 0.8668), (focu...","[(irrigation training research, 0.8668), (irri...","[0.007815127, -0.02983913, -0.041457575, 0.042...","[[-0.0008954684, -0.008144211, -0.015136938, 0...","[[0.0048553078, -0.008872252, -0.031875487, 0....","[[0.0048553078, -0.008872252, -0.031875487, 0...."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,2786497.txt,Texas A&amp;M Horticultural Extension - Vegeta...,Agriculture,"[(extension vegetables, 0.6118), (newsletters ...","[(vegetable grower handbook, 0.7008), (newslet...","[(vegetable grower handbook, 0.7008), (newslet...","[0.016607286, -0.03996504, 0.01082185, 0.05656...","[[-0.03316761, 0.004545155, -0.05236538, 0.022...","[[-0.079359375, -0.014674136, -0.07064848, -0....","[[-0.079359375, -0.014674136, -0.07064848, -0....","[[0.016607286, -0.03996504, 0.01082185, 0.0565..."
1,1,2784741.txt,Algaculture Information from Wikipedia on this...,Agriculture,"[(production microalgae, 0.7194), (algaculture...","[(algaculture information wikipedia, 0.7318), ...","[(algaculture information wikipedia, 0.7318), ...","[-0.052518472, 0.07755526, -0.09522597, 0.0270...","[[-0.035396907, -0.001984206, -0.060108755, -0...","[[0.0007872191, 0.027178274, -0.08233266, 0.03...","[[0.0007872191, 0.027178274, -0.08233266, 0.03...","[[-0.052518472, 0.07755526, -0.09522597, 0.027..."
2,2,2785016.txt,Annual Canarygrass Factsheet on this grain cro...,Agriculture,"[(annual canarygrass, 0.7755), (canarygrass fa...","[(annual canarygrass factsheet, 0.7986), (cana...","[(annual canarygrass factsheet, 0.7986), (annu...","[0.03832742, 0.018079218, -0.039859474, 0.0059...","[[0.013984093, 0.0032539438, 0.012409236, 0.00...","[[0.030801402, 0.013365954, -0.010543321, 0.01...","[[0.030801402, 0.013365954, -0.010543321, 0.01...","[[0.03832742, 0.018079218, -0.039859474, 0.005..."
3,3,2786133.txt,Black Sigatoka Photographs and information on ...,Agriculture,"[(disease bananas, 0.681), (fungal disease, 0....","[(fungal disease bananas, 0.8024), (disease ba...","[(fungal disease bananas, 0.8024), (disease ba...","[-0.0030358238, 0.058499087, -0.08342932, -0.0...","[[-0.034461573, 0.0020409657, -0.01293984, 0.0...","[[-0.018461134, 0.02032768, -0.049900673, -0.0...","[[-0.018461134, 0.02032768, -0.049900673, -0.0...","[[-0.0030358238, 0.058499087, -0.08342932, -0...."
4,4,2786989.txt,Irrigation Training and Research Center Univer...,Agriculture,"[(irrigation training, 0.8181), (focusing irri...","[(irrigation training research, 0.8668), (focu...","[(irrigation training research, 0.8668), (irri...","[0.007815127, -0.02983913, -0.041457575, 0.042...","[[-0.0008954684, -0.008144211, -0.015136938, 0...","[[0.0048553078, -0.008872252, -0.031875487, 0....","[[0.0048553078, -0.008872252, -0.031875487, 0....","[[-0.00076061627, -0.034473997, -0.05131875, 0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# classic4

In [ ]:
dataset_name = "classic4"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,cacm.001232,Representation of the Standard ECMA 7-Bit Code...,cacm,"[(cards ecma, 0.6013), (standard ecma, 0.5788)...","[(ecma bit code, 0.7377), (punched cards ecma,...","[(ecma bit code, 0.7377), (punched cards ecma,...","[-0.020285996, 0.10625108, -0.076215506, 0.018...","[[-0.045665547, 0.12378346, -0.046801522, 0.03...","[[-0.0124916835, 0.112781115, -0.07126342, -0....","[[-0.0124916835, 0.112781115, -0.07126342, -0...."
1,1,cacm.002901,An Encoding Method for Multifield Sorting and ...,cacm,"[(strings order, 0.6238), (multifield sorting,...","[(character strings order, 0.6571), (order pre...","[(character strings order, 0.6571), (order pre...","[-0.011131115, 0.019999074, 0.022867514, -0.09...","[[0.030976735, 0.0008268596, 0.04783698, -0.07...","[[0.019095283, -0.042498726, 0.07225646, -0.04...","[[0.019095283, -0.042498726, 0.07225646, -0.04..."
2,2,cacm.003171,Line Numbers Made Cheap A technique is describ...,cacm,"[(line numbers, 0.6122), (line number, 0.5592)...","[(line number administration, 0.6411), (run ti...","[(line number administration, 0.6411), (line n...","[-0.09843491, 0.07957201, -0.055119168, -0.044...","[[-0.10927536, 0.013597114, -0.07363706, -0.04...","[[-0.13844566, -0.02322, -0.068612196, -0.0437...","[[-0.13844566, -0.02322, -0.068612196, -0.0437..."
3,3,cacm.001938,Some Criteria for Time-Sharing System Performa...,cacm,"[(time sharing, 0.6189), (sharing performance,...","[(performance time sharing, 0.7735), (time sha...","[(performance time sharing, 0.7735), (time sha...","[0.023541665, 0.021506364, -0.046768464, -0.09...","[[-0.07385589, 0.006236384, -0.010160485, -0.0...","[[-0.022445958, 0.013442152, -0.07100275, -0.0...","[[-0.022445958, 0.013442152, -0.07100275, -0.0..."
4,4,cacm.000233,Abstracts-Additional Nuclear Reactor Codes,cacm,"[(reactor codes, 0.8534), (nuclear reactor, 0....","[(nuclear reactor codes, 0.9031), (additional ...","[(nuclear reactor codes, 0.9031), (reactor cod...","[-0.07201461, -0.018276004, -0.061757967, 0.06...","[[-0.093710095, -0.011074142, -0.043316126, 0....","[[-0.062427666, 0.00642693, -0.08581107, 0.059...","[[-0.062427666, 0.00642693, -0.08581107, 0.059..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,cacm.001232,Representation of the Standard ECMA 7-Bit Code...,cacm,"[(cards ecma, 0.6013), (standard ecma, 0.5788)...","[(ecma bit code, 0.7377), (punched cards ecma,...","[(ecma bit code, 0.7377), (punched cards ecma,...","[-0.020285996, 0.10625108, -0.076215506, 0.018...","[[-0.045665547, 0.12378346, -0.046801522, 0.03...","[[-0.0124916835, 0.112781115, -0.07126342, -0....","[[-0.0124916835, 0.112781115, -0.07126342, -0....","[[-0.020285996, 0.10625108, -0.076215506, 0.01..."
1,1,cacm.002901,An Encoding Method for Multifield Sorting and ...,cacm,"[(strings order, 0.6238), (multifield sorting,...","[(character strings order, 0.6571), (order pre...","[(character strings order, 0.6571), (order pre...","[-0.011131115, 0.019999074, 0.022867514, -0.09...","[[0.030976735, 0.0008268596, 0.04783698, -0.07...","[[0.019095283, -0.042498726, 0.07225646, -0.04...","[[0.019095283, -0.042498726, 0.07225646, -0.04...","[[-0.01558414, -0.002792653, 0.0073925816, -0...."
2,2,cacm.003171,Line Numbers Made Cheap A technique is describ...,cacm,"[(line numbers, 0.6122), (line number, 0.5592)...","[(line number administration, 0.6411), (run ti...","[(line number administration, 0.6411), (line n...","[-0.09843491, 0.07957201, -0.055119168, -0.044...","[[-0.10927536, 0.013597114, -0.07363706, -0.04...","[[-0.13844566, -0.02322, -0.068612196, -0.0437...","[[-0.13844566, -0.02322, -0.068612196, -0.0437...","[[-0.11782935, 0.05442667, -0.05098216, -0.058..."
3,3,cacm.001938,Some Criteria for Time-Sharing System Performa...,cacm,"[(time sharing, 0.6189), (sharing performance,...","[(performance time sharing, 0.7735), (time sha...","[(performance time sharing, 0.7735), (time sha...","[0.023541665, 0.021506364, -0.046768464, -0.09...","[[-0.07385589, 0.006236384, -0.010160485, -0.0...","[[-0.022445958, 0.013442152, -0.07100275, -0.0...","[[-0.022445958, 0.013442152, -0.07100275, -0.0...","[[0.0143550495, 0.0071380404, -0.07452241, -0...."
4,4,cacm.000233,Abstracts-Additional Nuclear Reactor Codes,cacm,"[(reactor codes, 0.8534), (nuclear reactor, 0....","[(nuclear reactor codes, 0.9031), (additional ...","[(nuclear reactor codes, 0.9031), (reactor cod...","[-0.07201461, -0.018276004, -0.061757967, 0.06...","[[-0.093710095, -0.011074142, -0.043316126, 0....","[[-0.062427666, 0.00642693, -0.08581107, 0.059...","[[-0.062427666, 0.00642693, -0.08581107, 0.059...","[[-0.07201461, -0.018276004, -0.061757967, 0.0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# review_polarity

In [ ]:
dataset_name = "review_polarity"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,cv145_12239.txt,it was once said that in order to truly enjoy ...,neg,"[(shawshank redemption, 0.4308), (overwhelms h...","[(huntingburg indiana film, 0.4553), (played t...","[(huntingburg indiana film, 0.4553), (played t...","[-0.03404502, -0.00043280784, 0.02967335, 0.05...","[[-0.07603545, -0.0022513822, -0.024598675, 0....","[[-0.05451088, 0.05801035, -0.07663032, 0.0188...","[[-0.05451088, 0.05801035, -0.07663032, 0.0188..."
1,1,cv745_14009.txt,conventional wisdom among collectibles retaile...,neg,"[(nostalgia nowadays, 0.4483), (yore hollywood...","[(toys late sixties, 0.5266), (heavy nostalgia...","[(toys late sixties, 0.5266), (heavy nostalgia...","[-0.08428222, 0.02908774, 0.08786607, -0.02479...","[[0.013637959, 0.031474095, -0.004310135, -0.0...","[[-0.026392736, 0.051586345, 0.0170478, 0.0051...","[[-0.026392736, 0.051586345, 0.0170478, 0.0051..."
2,2,cv376_20883.txt,robin williams has the rarest of gifts : the a...,neg,"[(robin williams, 0.509), (jakob liar, 0.5062)...","[(jakob liar forgets, 0.5045), (jakob ghetto l...","[(robin williams, 0.509), (jakob liar, 0.5062)...","[-0.0931352, 0.00610449, -0.014688251, -0.0120...","[[-0.056007113, -0.031148888, -0.051129047, 0....","[[-0.062197305, -0.05801207, 0.039777476, -0.0...","[[-0.056007113, -0.031148888, -0.051129047, 0...."
3,3,cv109_22599.txt,nearly every film tim burton has directed has ...,neg,"[(sleepy hollow, 0.6049), (hollow movie, 0.548...","[(sleepy hollow movie, 0.6832), (hollow movie ...","[(sleepy hollow movie, 0.6832), (hollow movie ...","[-0.065481186, 0.056043383, 0.015037637, 0.036...","[[0.024425134, 0.01257685, -0.011018157, 0.047...","[[-0.028323261, 0.046399403, -0.045313556, 0.0...","[[-0.028323261, 0.046399403, -0.045313556, 0.0..."
4,4,cv494_18689.txt,"you know , i never really wondered what the ta...",neg,"[(movies tarzan, 0.5787), (tarzan films, 0.574...","[(films like tarzan, 0.6003), (wondered tarzan...","[(films like tarzan, 0.6003), (wondered tarzan...","[-0.10274353, 0.013595925, -0.022167694, -0.04...","[[-0.116359934, -0.0050691105, -0.0728453, 0.0...","[[-0.087046064, -0.028026031, -0.046432335, 0....","[[-0.087046064, -0.028026031, -0.046432335, 0...."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,cv145_12239.txt,it was once said that in order to truly enjoy ...,neg,"[(shawshank redemption, 0.4308), (overwhelms h...","[(huntingburg indiana film, 0.4553), (played t...","[(huntingburg indiana film, 0.4553), (played t...","[-0.03404502, -0.00043280784, 0.02967335, 0.05...","[[-0.07603545, -0.0022513822, -0.024598675, 0....","[[-0.05451088, 0.05801035, -0.07663032, 0.0188...","[[-0.05451088, 0.05801035, -0.07663032, 0.0188...","[[0.01957459, -0.06495593, 0.013693364, 0.0351..."
1,1,cv745_14009.txt,conventional wisdom among collectibles retaile...,neg,"[(nostalgia nowadays, 0.4483), (yore hollywood...","[(toys late sixties, 0.5266), (heavy nostalgia...","[(toys late sixties, 0.5266), (heavy nostalgia...","[-0.08428222, 0.02908774, 0.08786607, -0.02479...","[[0.013637959, 0.031474095, -0.004310135, -0.0...","[[-0.026392736, 0.051586345, 0.0170478, 0.0051...","[[-0.026392736, 0.051586345, 0.0170478, 0.0051...","[[-0.07162021, 0.051433988, 0.011552863, 0.032..."
2,2,cv376_20883.txt,robin williams has the rarest of gifts : the a...,neg,"[(robin williams, 0.509), (jakob liar, 0.5062)...","[(jakob liar forgets, 0.5045), (jakob ghetto l...","[(robin williams, 0.509), (jakob liar, 0.5062)...","[-0.0931352, 0.00610449, -0.014688251, -0.0120...","[[-0.056007113, -0.031148888, -0.051129047, 0....","[[-0.062197305, -0.05801207, 0.039777476, -0.0...","[[-0.056007113, -0.031148888, -0.051129047, 0....","[[-0.109135, -0.016573096, -0.00738937, 0.0147..."
3,3,cv109_22599.txt,nearly every film tim burton has directed has ...,neg,"[(sleepy hollow, 0.6049), (hollow movie, 0.548...","[(sleepy hollow movie, 0.6832), (hollow movie ...","[(sleepy hollow movie, 0.6832), (hollow movie ...","[-0.065481186, 0.056043383, 0.015037637, 0.036...","[[0.024425134, 0.01257685, -0.011018157, 0.047...","[[-0.028323261, 0.046399403, -0.045313556, 0.0...","[[-0.028323261, 0.046399403, -0.045313556, 0.0...","[[-0.061315138, -0.03929636, -0.029498795, 0.0..."
4,4,cv494_18689.txt,"you know , i never really wondered what the ta...",neg,"[(movies tarzan, 0.5787), (tarzan films, 0.574...","[(films like tarzan, 0.6003), (wondered tarzan...","[(films like tarzan, 0.6003), (wondered tarzan...","[-0.10274353, 0.013595925, -0.022167694, -0.04...","[[-0.116359934, -0.0050691105, -0.0728453, 0.0...","[[-0.087046064, -0.028026031, -0.046432335, 0....","[[-0.087046064, -0.028026031, -0.046432335, 0....","[[-0.051731523, 0.04584322, -0.034292366, 0.02..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# SyskillWebert

In [ ]:
dataset_name = "SyskillWebert"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,53,"LaFond, Lois One World (...",Bands,"[(songs lafond, 0.5778), (rockadile music, 0.5...","[(lafond rockadiles songs, 0.6489), (world chi...","[(lafond rockadiles songs, 0.6489), (world chi...","[-0.0077898204, -0.018726388, -0.036333524, -0...","[[0.0063638324, -0.102769606, -0.02681988, -0....","[[0.015110715, -0.04565199, -0.018088073, -0.0...","[[0.015110715, -0.04565199, -0.018088073, -0.0..."
1,1,28,"Houston, Penelope Houston, Penelope ...",Bands,"[(penelope houston, 0.6083), (houston penelope...","[(music avengers penelope, 0.6799), (band aven...","[(music avengers penelope, 0.6799), (band aven...","[0.020465637, -0.15591253, 0.012023962, -0.055...","[[-0.07084119, -0.09657672, -0.0028063285, 0.0...","[[-0.056443516, -0.10189217, -0.008211225, -0....","[[-0.056443516, -0.10189217, -0.008211225, -0...."
2,2,57,"Bernard, Mary Ellen Bernard, Mary Ellen ...",Bands,"[(latvia listeners, 0.529), (radio riga, 0.457...","[(seattle radio riga, 0.5571), (yes latvia lis...","[(seattle radio riga, 0.5571), (latvia listene...","[0.020895522, -0.09832781, 0.017503079, -0.052...","[[-1.2988382e-05, -0.05162534, -0.02839006, -0...","[[-0.0048518856, 0.0060428, -0.0774081, 0.0020...","[[-0.0048518856, 0.0060428, -0.0774081, 0.0020..."
3,3,25,Deth Specula Deth Specula Careenin...,Bands,"[(deth specula, 0.5982), (specula deth, 0.5812...","[(deth specula fans, 0.5988), (updated deth sp...","[(deth specula fans, 0.5988), (deth specula, 0...","[-0.08653825, -0.082185246, -0.01824997, -0.03...","[[-0.043629605, 0.0023226792, -0.013162603, -0...","[[-0.03944687, -0.041941278, 0.02111801, -0.06...","[[-0.03944687, -0.041941278, 0.02111801, -0.06..."
4,4,15,Hungry Ghost Hungry Ghost Aboral ...,Bands,"[(ghost hungry, 0.6302), (hungry ghost, 0.6258...","[(hungry ghost aboral, 0.6592), (hungry ghost ...","[(hungry ghost aboral, 0.6592), (hungry ghost ...","[-0.002208711, -0.03985782, -0.025887983, 0.02...","[[-0.05560613, 0.021909092, -0.010372226, 0.01...","[[-0.054047257, -0.0182318, 0.005286151, -0.00...","[[-0.054047257, -0.0182318, 0.005286151, -0.00..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,53,"LaFond, Lois One World (...",Bands,"[(songs lafond, 0.5778), (rockadile music, 0.5...","[(lafond rockadiles songs, 0.6489), (world chi...","[(lafond rockadiles songs, 0.6489), (world chi...","[-0.0077898204, -0.018726388, -0.036333524, -0...","[[0.0063638324, -0.102769606, -0.02681988, -0....","[[0.015110715, -0.04565199, -0.018088073, -0.0...","[[0.015110715, -0.04565199, -0.018088073, -0.0...","[[0.018322192, 0.0077181645, -0.036671143, -0...."
1,1,28,"Houston, Penelope Houston, Penelope ...",Bands,"[(penelope houston, 0.6083), (houston penelope...","[(music avengers penelope, 0.6799), (band aven...","[(music avengers penelope, 0.6799), (band aven...","[0.020465637, -0.15591253, 0.012023962, -0.055...","[[-0.07084119, -0.09657672, -0.0028063285, 0.0...","[[-0.056443516, -0.10189217, -0.008211225, -0....","[[-0.056443516, -0.10189217, -0.008211225, -0....","[[0.002881291, -0.13499507, -0.03816083, -0.02..."
2,2,57,"Bernard, Mary Ellen Bernard, Mary Ellen ...",Bands,"[(latvia listeners, 0.529), (radio riga, 0.457...","[(seattle radio riga, 0.5571), (yes latvia lis...","[(seattle radio riga, 0.5571), (latvia listene...","[0.020895522, -0.09832781, 0.017503079, -0.052...","[[-1.2988382e-05, -0.05162534, -0.02839006, -0...","[[-0.0048518856, 0.0060428, -0.0774081, 0.0020...","[[-0.0048518856, 0.0060428, -0.0774081, 0.0020...","[[-0.029643955, -0.057621337, 0.04576483, -0.0..."
3,3,25,Deth Specula Deth Specula Careenin...,Bands,"[(deth specula, 0.5982), (specula deth, 0.5812...","[(deth specula fans, 0.5988), (updated deth sp...","[(deth specula fans, 0.5988), (deth specula, 0...","[-0.08653825, -0.082185246, -0.01824997, -0.03...","[[-0.043629605, 0.0023226792, -0.013162603, -0...","[[-0.03944687, -0.041941278, 0.02111801, -0.06...","[[-0.03944687, -0.041941278, 0.02111801, -0.06...","[[-0.0015062785, -0.060788594, 0.037033446, 0...."
4,4,15,Hungry Ghost Hungry Ghost Aboral ...,Bands,"[(ghost hungry, 0.6302), (hungry ghost, 0.6258...","[(hungry ghost aboral, 0.6592), (hungry ghost ...","[(hungry ghost aboral, 0.6592), (hungry ghost ...","[-0.002208711, -0.03985782, -0.025887983, 0.02...","[[-0.05560613, 0.021909092, -0.010372226, 0.01...","[[-0.054047257, -0.0182318, 0.005286151, -0.00...","[[-0.054047257, -0.0182318, 0.005286151, -0.00...","[[-0.03327608, -0.036834795, 0.038693734, -0.0..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')

# webkb_parsed

In [ ]:
dataset_name = "webkb_parsed"

## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings
0,0,http:^^www.cs.utexas.edu^users^dahlin^Classes^...,Content-Type: text/html Content-Length: 1978 L...,course,"[(web operating, 0.6921), (web related, 0.535)...","[(web operating systems, 0.7051), (purpose web...","[(web operating systems, 0.7051), (purpose web...","[-0.0927243, 0.0012164878, -0.05404062, -0.038...","[[-0.036058955, -0.01910096, -0.044949748, -0....","[[-0.045564666, -0.028437821, -0.032989483, -0...","[[-0.045564666, -0.028437821, -0.032989483, -0..."
1,1,http:^^www.cs.rutgers.edu^~venugopa^112^112.html,CS 112: Data Structures CS 112: Data Struc...,course,"[(programming unix, 0.6077), (programming assi...","[(data structures cs, 0.6183), (programming as...","[(data structures cs, 0.6183), (programming un...","[-0.0030399715, 0.039208196, -0.009568916, -0....","[[-0.019407477, 0.030763928, -0.07262235, -0.0...","[[-0.036686115, 0.036067266, -0.03149169, 0.00...","[[-0.036686115, 0.036067266, -0.03149169, 0.00..."
2,2,http:^^www.eecs.umich.edu^courses^eecs661^eecs...,Set-Cookie: Apache=gs359343849035331353; path=...,course,"[(event systems, 0.5776), (systems eecs, 0.524...","[(discrete event systems, 0.6734), (event dyna...","[(discrete event systems, 0.6734), (event dyna...","[-0.07351037, -0.0034346038, -0.021328399, -0....","[[-0.022722067, 0.0026163666, 0.0029694512, -0...","[[-0.04119877, 0.008551315, -0.004188435, -0.0...","[[-0.04119877, 0.008551315, -0.004188435, -0.0..."
3,3,http:^^www.cis.upenn.edu^~cse110e^cse110e.html,CSE 110 (Accelerated) Home Page CSE 110 (Acc...,course,"[(cse 110, 0.6049), (page cse, 0.5217), (intro...","[(cse 110 accelerated, 0.6174), (page cse 110,...","[(cse 110 accelerated, 0.6174), (cse 110, 0.60...","[-0.13189143, 0.035983704, 0.011182002, -0.010...","[[-0.12709178, 0.034261793, -0.008630871, 0.00...","[[-0.105777584, 0.0013491624, 0.007880913, 0.0...","[[-0.105777584, 0.0013491624, 0.007880913, 0.0..."
4,4,http:^^www.CS.Princeton.EDU^courses^archive^fa...,COS 320: Compiling Techniques. Fall 1995 ...,course,"[(computers grading, 0.5084), (programming ass...","[(cos 320 compiling, 0.5812), (computers gradi...","[(cos 320 compiling, 0.5812), (computers gradi...","[-0.10675096, -0.00079041265, -0.08070004, -0....","[[-0.030724177, 0.008653546, -0.05521948, -0.0...","[[-0.08890971, 0.05575665, -0.1463867, -0.0679...","[[-0.08890971, 0.05575665, -0.1463867, -0.0679..."


## Separando sentenças e seus embedings

In [ ]:
sentences_embeddings = []

for i in range(len(df)):
    sentences = sent_tokenize(df["text"][i])

    s_embeddings = []

    for j in range(len(sentences)):
        s_embeddings.append(model.encode(sentences[j]))

    sentences_embeddings.append(s_embeddings)

df['sentences_embeddings'] = sentences_embeddings

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,http:^^www.cs.utexas.edu^users^dahlin^Classes^...,Content-Type: text/html Content-Length: 1978 L...,course,"[(web operating, 0.6921), (web related, 0.535)...","[(web operating systems, 0.7051), (purpose web...","[(web operating systems, 0.7051), (purpose web...","[-0.0927243, 0.0012164878, -0.05404062, -0.038...","[[-0.036058955, -0.01910096, -0.044949748, -0....","[[-0.045564666, -0.028437821, -0.032989483, -0...","[[-0.045564666, -0.028437821, -0.032989483, -0...","[[-0.010800143, 0.014000101, -0.044020705, -0...."
1,1,http:^^www.cs.rutgers.edu^~venugopa^112^112.html,CS 112: Data Structures CS 112: Data Struc...,course,"[(programming unix, 0.6077), (programming assi...","[(data structures cs, 0.6183), (programming as...","[(data structures cs, 0.6183), (programming un...","[-0.0030399715, 0.039208196, -0.009568916, -0....","[[-0.019407477, 0.030763928, -0.07262235, -0.0...","[[-0.036686115, 0.036067266, -0.03149169, 0.00...","[[-0.036686115, 0.036067266, -0.03149169, 0.00...","[[-0.0030399715, 0.039208196, -0.009568916, -0..."
2,2,http:^^www.eecs.umich.edu^courses^eecs661^eecs...,Set-Cookie: Apache=gs359343849035331353; path=...,course,"[(event systems, 0.5776), (systems eecs, 0.524...","[(discrete event systems, 0.6734), (event dyna...","[(discrete event systems, 0.6734), (event dyna...","[-0.07351037, -0.0034346038, -0.021328399, -0....","[[-0.022722067, 0.0026163666, 0.0029694512, -0...","[[-0.04119877, 0.008551315, -0.004188435, -0.0...","[[-0.04119877, 0.008551315, -0.004188435, -0.0...","[[-0.015701072, -0.001414385, -0.08331965, -0...."
3,3,http:^^www.cis.upenn.edu^~cse110e^cse110e.html,CSE 110 (Accelerated) Home Page CSE 110 (Acc...,course,"[(cse 110, 0.6049), (page cse, 0.5217), (intro...","[(cse 110 accelerated, 0.6174), (page cse 110,...","[(cse 110 accelerated, 0.6174), (cse 110, 0.60...","[-0.13189143, 0.035983704, 0.011182002, -0.010...","[[-0.12709178, 0.034261793, -0.008630871, 0.00...","[[-0.105777584, 0.0013491624, 0.007880913, 0.0...","[[-0.105777584, 0.0013491624, 0.007880913, 0.0...","[[-0.15982199, 0.02581229, -0.0061420402, -0.0..."
4,4,http:^^www.CS.Princeton.EDU^courses^archive^fa...,COS 320: Compiling Techniques. Fall 1995 ...,course,"[(computers grading, 0.5084), (programming ass...","[(cos 320 compiling, 0.5812), (computers gradi...","[(cos 320 compiling, 0.5812), (computers gradi...","[-0.10675096, -0.00079041265, -0.08070004, -0....","[[-0.030724177, 0.008653546, -0.05521948, -0.0...","[[-0.08890971, 0.05575665, -0.1463867, -0.0679...","[[-0.08890971, 0.05575665, -0.1463867, -0.0679...","[[-0.10500991, 0.087999925, -0.14258686, -0.06..."


In [ ]:
df.to_pickle('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/'+dataset_name+'.csv')